## Annother try to predict stock pris by decision tree
### Vol. 1
#### 30.06.19

In [1]:
# connecting nessesary paceges
from pandas import read_csv as read
from pandas import DataFrame as DF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.tree import export_graphviz
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.display import SVG
from IPython.display import display
from graphviz import Source
from IPython.display import HTML
style = "<style>svg{width:80% !important;height:80% !important;}</style>"
HTML(style)

In [2]:
# load data from github
gazp = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/GAZP.txt')
gazp = gazp.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)
sber = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/SBER.txt')
sber = sber.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)
luko = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/LKOH.txt')
luko = luko.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)

In [3]:
gazp.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,239.0,239.00,218.49,218.89,5078252
1,220.5,224.68,219.66,224.00,8971078
2,225.2,231.00,225.00,228.38,15467697
3,228.9,229.41,223.51,224.47,7585458
4,226.2,231.50,224.00,228.75,12719299


In [4]:
sber.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,1112.0,1185.0,1112.0,1180.0,12261
1,1190.0,1260.0,1166.0,1250.0,16377
2,1251.0,1266.0,1191.1,1211.0,30724
3,1200.0,1214.0,1123.0,1129.0,18110
4,1111.1,1188.0,1090.0,1185.0,23418


In [5]:
luko.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,390.95,400.0,385.5,397.0,93790
1,398.01,412.5,389.0,410.0,61148
2,406.00,427.0,405.0,415.0,199094
3,412.00,416.0,390.6,397.5,2227709
4,386.00,400.0,380.0,398.0,154340


## Transform data to boolean variables

In [6]:
gazp_bool = gazp.apply(lambda x: x - x.shift(1))
gazp_bool = gazp_bool.applymap(lambda x: 1 if x > 0 else 0)

In [7]:
sber_bool = sber.apply(lambda x: x - x.shift(1))
sber_bool = sber_bool.applymap(lambda x: 1 if x > 0 else 0)

In [8]:
luko_bool = luko.apply(lambda x: x - x.shift(1))
luko_bool = luko_bool.applymap(lambda x: 1 if x > 0 else 0)

## Get 1 if open price upper that yestarday close price

In [9]:
gazp_grow = [float('nan')]
for i in range(1,len(gazp.index)):
    if gazp.loc[i, '<OPEN>'] > gazp.loc[i-1, '<CLOSE>']:
        gazp_grow.append(1)
    else:
        gazp_grow.append(0)

In [10]:
sber_grow = [float('nan')]
for i in range(1,len(sber.index)):
    if sber.loc[i, '<OPEN>'] > sber.loc[i-1, '<CLOSE>']:
        sber_grow.append(1)
    else:
        sber_grow.append(0)

In [11]:
luko_grow = [float('nan')]
for i in range(1,len(luko.index)):
    if luko.loc[i, '<OPEN>'] > luko.loc[i-1, '<CLOSE>']:
        luko_grow.append(1)
    else:
        luko_grow.append(0)

In [12]:
gazp_bool['grow'] = gazp_grow
sber_bool['grow'] = sber_grow
luko_bool['grow'] = luko_grow
gazp_bool = gazp_bool.dropna()
sber_bool = sber_bool.dropna()
luko_bool = luko_bool.dropna()

In [13]:
gazp_bool = gazp_bool.applymap(int)
gazp_bool.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,grow
1,0,0,1,1,1,1
2,1,1,1,1,1,1
3,1,0,0,0,0,1
4,0,1,1,1,1,1
5,1,0,0,0,1,1


In [14]:
sber_bool = sber_bool.applymap(int)
sber_bool.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,grow
1,1,1,1,1,1,1
2,1,1,1,0,1,1
3,0,0,0,0,0,0
4,0,0,0,1,1,0
5,1,1,1,1,0,1


In [15]:
luko_bool = luko_bool.applymap(int)
luko_bool.head()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,grow
1,1,1,1,1,0,1
2,1,1,1,1,1,0
3,1,0,0,0,1,0
4,0,0,0,1,0,0
5,1,1,1,1,0,1


In [16]:
gazp_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(gazp_bool):
    train = gazp_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(gazp_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
gazp_result['predict'] = calc
gazp_result['fact'] = fact
gazp_result['hits'] = gazp_result['predict'] == gazp_result['fact']
gazp_result.hits.sum() / len(gazp_result)

0.6519337016574586

In [17]:
sber_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(sber_bool):
    train = sber_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(sber_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
sber_result['predict'] = calc
sber_result['fact'] = fact
sber_result['hits'] = sber_result['predict'] == sber_result['fact']
sber_result.hits.sum() / len(sber_result)

0.6637282304880671

In [18]:
luko_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(luko_bool):
    train = luko_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(luko_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
luko_result['predict'] = calc
luko_result['fact'] = fact
luko_result['hits'] = luko_result['predict'] == luko_result['fact']
luko_result.hits.sum() / len(luko_result)

0.621059403817285

## As we can see, the probability of getting a little more than 60%. Let's try to look at the probability of hitting the strategy, taking into account the excess of the opening of today over the closing of the previous one

In [19]:
gazp_result[gazp_result.fact > 0].hits.sum() / len(gazp_result[gazp_result.fact > 0])

0.6668505239933812

In [20]:
sber_result[sber_result.fact > 0].hits.sum() / len(sber_result[sber_result.fact > 0])

0.6715553840216133

In [21]:
luko_result[luko_result.fact > 0].hits.sum() / len(luko_result[luko_result.fact > 0])

0.6186174391201885

## The chances of getting hit have not changed.
## Let's test the strategy on data with a frame of 1 min

In [23]:
# load data from github
gazp = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/GAZP_min.txt')
gazp = gazp.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)
sber = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/SBER_min.txt')
sber = sber.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)
luko = read('https://raw.githubusercontent.com/Aurilino/Python/master/Annother_decision_tree/LKOH_min.txt')
luko = luko.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1)

In [24]:
luko_bool = luko.apply(lambda x: x - x.shift(1))
luko_bool = luko_bool.applymap(lambda x: 1 if x > 0 else 0)
gazp_bool = gazp.apply(lambda x: x - x.shift(1))
gazp_bool = gazp_bool.applymap(lambda x: 1 if x > 0 else 0)
sber_bool = sber.apply(lambda x: x - x.shift(1))
sber_bool = sber_bool.applymap(lambda x: 1 if x > 0 else 0)

In [25]:
gazp_grow = [float('nan')]
for i in range(1,len(gazp.index)):
    if gazp.loc[i, '<OPEN>'] > gazp.loc[i-1, '<CLOSE>']:
        gazp_grow.append(1)
    else:
        gazp_grow.append(0)
sber_grow = [float('nan')]
for i in range(1,len(sber.index)):
    if sber.loc[i, '<OPEN>'] > sber.loc[i-1, '<CLOSE>']:
        sber_grow.append(1)
    else:
        sber_grow.append(0)
luko_grow = [float('nan')]
for i in range(1,len(luko.index)):
    if luko.loc[i, '<OPEN>'] > luko.loc[i-1, '<CLOSE>']:
        luko_grow.append(1)
    else:
        luko_grow.append(0)

In [26]:
gazp_bool['grow'] = gazp_grow
sber_bool['grow'] = sber_grow
luko_bool['grow'] = luko_grow
gazp_bool = gazp_bool.dropna()
sber_bool = sber_bool.dropna()
luko_bool = luko_bool.dropna()
gazp_bool = gazp_bool.applymap(int)
sber_bool = sber_bool.applymap(int)
luko_bool = luko_bool.applymap(int)

In [27]:
gazp_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(gazp_bool):
    train = gazp_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(gazp_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
gazp_result['predict'] = calc
gazp_result['fact'] = fact
gazp_result['hits'] = gazp_result['predict'] == gazp_result['fact']
gazp_result.hits.sum() / len(gazp_result)

0.66390631125049

In [28]:
sber_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(sber_bool):
    train = sber_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(sber_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
sber_result['predict'] = calc
sber_result['fact'] = fact
sber_result['hits'] = sber_result['predict'] == sber_result['fact']
sber_result.hits.sum() / len(sber_result)

0.6421208859314349

In [29]:
luko_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(luko_bool):
    train = luko_bool.iloc[start:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(luko_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
luko_result['predict'] = calc
luko_result['fact'] = fact
luko_result['hits'] = luko_result['predict'] == luko_result['fact']
luko_result.hits.sum() / len(luko_result)

0.7035676637238107

In [30]:
print(gazp_result[gazp_result.fact > 0].hits.sum() / len(gazp_result[gazp_result.fact > 0]))
print(sber_result[sber_result.fact > 0].hits.sum() / len(sber_result[sber_result.fact > 0]))
print(luko_result[luko_result.fact > 0].hits.sum() / len(luko_result[luko_result.fact > 0]))

0.2840765617898086
0.21695974202650276
0.4213914335231493


## As we can see, chance to hit consist 28-42% with the opening of today over the closing of the previous one

## Remove the lower limit for tree training

In [31]:
gazp_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(gazp_bool):
    train = gazp_bool.iloc[:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(gazp_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
gazp_result['predict'] = calc
gazp_result['fact'] = fact
gazp_result['hits'] = gazp_result['predict'] == gazp_result['fact']
gazp_result.hits.sum() / len(gazp_result)

0.6932412125963674

In [ ]:
sber_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(sber_bool):
    train = sber_bool.iloc[:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(sber_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
sber_result['predict'] = calc
sber_result['fact'] = fact
sber_result['hits'] = sber_result['predict'] == sber_result['fact']
sber_result.hits.sum() / len(sber_result)

0.678637831490361

In [ ]:
luko_result = DF()
clf = Tree(criterion='entropy')
step = 100
start = 0
finish = start + step
calc = []
fact = []
while finish < len(luko_bool):
    train = luko_bool.iloc[:finish, :]
    x_train = train.drop('grow', axis=1)
    y_train = train.grow
    clf.fit(x_train, y_train)
    test = DF(luko_bool.iloc[finish, :]).T
    x_test = test.drop('grow', axis=1)
    y_test = test.grow
    calc.append(int(clf.predict(x_test)))
    fact.append(int(y_test))
    finish += 1
    start += 1
luko_result['predict'] = calc
luko_result['fact'] = fact
luko_result['hits'] = luko_result['predict'] == luko_result['fact']
luko_result.hits.sum() / len(luko_result)

In [ ]:
print(gazp_result[gazp_result.fact > 0].hits.sum() / len(gazp_result[gazp_result.fact > 0]))
print(sber_result[sber_result.fact > 0].hits.sum() / len(sber_result[sber_result.fact > 0]))
print(luko_result[luko_result.fact > 0].hits.sum() / len(luko_result[luko_result.fact > 0]))

## The results have improved slightly

## Summary

### Prediction probabilities of exceeding prices open above the previous closing prices to an average of 60-70%

# In the next part will try to make a portfolio based on the relationship of the results of predictions